# Laplace

Notebook to perform Laplace transformations

Author: Lucas Schneider

---

## Initialization

In [ ]:
import sympy as sym
import numpy as np
#import oct2py

from sympy.integrals import laplace_transform
from sympy.integrals import inverse_laplace_transform

# sym.init_printing()

In [ ]:
t_var = sym.symbols('t', real=True)
s_var = sym.symbols('s')

## Direct Transform

In [ ]:
# Other symbols declaration
phi = sym.symbols('φ', real=True)

In [ ]:
# Input
expression = sym.cos(t_var)

expression *= sym.Heaviside(t_var)
print(str(expression).replace('**','^'))
expression

In [ ]:
U = laplace_transform(expression, t_var, s_var)
U[0]

## Inverse Transform

In [ ]:
pol = (((s_var - (1 + 2*sym.I))*(s_var - (1 - 2*sym.I))) ** 2)*(s_var-1)*(s_var-(2+sym.I))*(s_var-(2-sym.I))
pol.expand()

In [ ]:
N_list = [1, 5, 4, 3, 1]
D_list = [1, 3, 2, 0]

N_pol = sym.Poly(N_list, s_var)
D_pol = sym.Poly(D_list, s_var)
F_raw = N_pol / D_pol
print(str(F_raw).replace('**','^'))
F_raw

In [ ]:
# Perform the partial fraction decomposition
sum_of_complete, sum_of_fraction = sym.div(N_pol, D_pol)

sum_of_fraction = sym.apart(sum_of_fraction / D_pol, full=True).doit()

fraction_terms = list(sum_of_fraction.args)
fraction_terms = [frac.simplify() for frac in fraction_terms]

complete_terms = [list(reversed(sum_of_complete.coeffs()))[i] * s_var ** i for i in range(len(sum_of_complete.coeffs()))]

print(str(sum(complete_terms) + sum(fraction_terms)).replace('**','^'))
sum(complete_terms) + sum(fraction_terms)

In [ ]:
# Get a list of poles and their multiplicity
poles = []
multi = []

roots_dict = sym.polys.polyroots.roots(D_pol)
for root in roots_dict.keys():
    for i in range(roots_dict[root]):
        poles.append(root)
        multi.append(i+1)
        
# print(poles)
# print(multi)

In [ ]:
# Get the residues for each pole
residues = []

for i in range(len(poles)):
    for frac in fraction_terms:
        possible_residue = (frac * ((s_var - poles[i]) ** multi[i])).simplify()
        if str(s_var) not in str(possible_residue):
            # print(possible_residue, poles[i], sep='   ')
            residues.append(possible_residue)
            
print(residues, poles, multi, sep='\n')

In [ ]:
def conjugate_already_seen(pole, multi, poles_seen):
    for pole_seen, multi_seen in poles_seen:
        if (pole.equals(pole_seen) or (pole.conjugate()).equals(pole_seen)) and (multi == multi_seen):
            return True

    return False

In [ ]:
f_terms = []

complex_poles_done = []
for i in range(len(residues)):
    if not (sym.im(poles[i])).equals(0):
        if not conjugate_already_seen(poles[i], multi[i], complex_poles_done):
            if sym.im(residues[i]) < 0:
                Ak = residues[i].conjugate()
                pk = poles[i].conjugate()
            else:
                Ak = residues[i]
                pk = poles[i]

            # t_var ** (multi[i] - 1) / sym.factorial(multi[i] - 1)
            term = sym.Mul(t_var ** (multi[i] - 1) / sym.factorial(multi[i] - 1), 2 * sym.sqrt(sym.re(Ak) ** 2 + sym.im(Ak) ** 2), sym.exp(sym.re(pk) * t_var), sym.cos(sym.Add(sym.im(pk)*t_var, sym.arg(Ak), evaluate=False)))#, evaluate=False)

            # Para conferir com o Wolfram
            # term_check = sym.Mul(residues[i] / sym.factorial(multi[i] - 1), t_var ** (multi[i] - 1), sym.exp(poles[i] * t_var), evaluate=False)
            # term_check += sym.Mul(residues[i+1] / sym.factorial(multi[i+1] - 1), t_var ** (multi[i+1] - 1), sym.exp(poles[i+1] * t_var), evaluate=False)
            # print(term_check)
            # print(str((term_check - term).simplify()).replace('I','i'))

            print(Ak, pk, multi[i], str(term).replace('exp', 'e^'), sep='\n')
            print('')

            complex_poles_done.append((poles[i], multi[i]))
        else:
            continue
    else:
        term = sym.Mul(residues[i] / sym.factorial(multi[i] - 1), t_var ** (multi[i] - 1), sym.exp(poles[i] * t_var), evaluate=False)
    
    # print(term)
    # if multi[i] == 2:
    f_terms.append(term)

print(str(sum(f_terms)).replace('**','^').replace('exp', 'e^'))
sum(f_terms)

In [ ]:
f_terms = []

for i in range(len(residues)):
    term = sym.Mul(residues[i] / sym.factorial(multi[i] - 1), t_var ** (multi[i] - 1), sym.exp(poles[i] * t_var), evaluate=False)
    # print(term)
    
    # if multi[i] == 2:
    f_terms.append(term)

print(str(sum(f_terms)).replace('**','^').replace('I', 'i').replace('exp', 'e^'))
sum(f_terms)

In [ ]:
# Apply the Inverse Laplace Transform from sympy
f_complete_terms = []
f_fraction_terms = []

for term in complete_terms:
    transform = inverse_laplace_transform(term, s_var, t_var)
    f_complete_terms.append(transform)

for term in fraction_terms:
    transform = inverse_laplace_transform(term, s_var, t_var)
    f_fraction_terms.append(transform)

f = sum(f_complete_terms) + sum(f_fraction_terms)
print(str(f).replace('**','^'))
f

In [ ]:
f.evalf(subs={t_var: 0.5})

### Older Version

In [ ]:
# oc = oct2py.Oct2Py()

In [ ]:
# [res, poles, poly_int, multi] = oc.residue(N_list, D_list, nout=4)
# res = res.reshape(res.size)
# poles = poles.reshape(poles.size)
# multi = multi.reshape(multi.size)
# # print(poly_int)
# if type(poly_int) == np.ndarray:
#     poly_int = poly_int.reshape(-1)
# else:
#     poly_int = np.array(poly_int, ndmin=1)

# # print(poly_int)

# F = 0
# F_int_terms = []
# F_res_terms = []
# # Add the quocient from the polynomial division
# if poly_int.size > 0:
#     F_int_terms = [poly_int[poly_int.size - i - 1] * s_var ** i for i in range(poly_int.size)]


# # Add the residue
# for i in range(res.size):
#     F_res_terms.append( res[i] / (s_var - poles[i]) ** int(multi[i]) )

# F = sum(F_int_terms) + sum(F_res_terms)

# F.simplify()
# F

In [ ]:
# String for Wolfram
# print(str(F).replace('I', 'i').replace('**', '^'))

In [ ]:
# f_int_terms = []
# f_res_terms = []

# for term in F_int_terms:
#     transform = inverse_laplace_transform(term, s_var, t_var)
#     f_int_terms.append(transform)

# for term in F_res_terms:
#     transform = inverse_laplace_transform(term, s_var, t_var)
#     f_res_terms.append(transform)

# f = sum(f_int_terms) + sum(f_res_terms)
# f